<a href="https://colab.research.google.com/github/AFalabella0/SincNet/blob/keras/KerasSpeakerRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install p7zip-full
!7za x /content/drive/MyDrive/Dati/archive.zip

In [5]:
import os
import shutil
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras

from pathlib import Path
from IPython.display import display, Audio

In [13]:
AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"
DATASET_ROOT = os.path.join(os.getcwd, "16000_pcm_speeches")

DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

VALID_SPLIT = 0.1
SHUFFLE_SEED = 43

SAMPLING_RATE = 16000
SCALE = 0.5

BATCH_SIZE = 128
EPOCHS = 100

In [15]:
#move files
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)

for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            continue
        elif folder in ["other", "_background_noise_"]:
            shutil.move(os.path.join(DATASET_ROOT, folder),
                        os.path.join(DATASET_NOISE_PATH, folder),
                        )
        else:
            shutil.move(os.path.join(DATASET_ROOT, folder),
                        os.path.join(DATASET_AUDIO_PATH, folder),
                        )
            

In [20]:
#get list of noise files

noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]
print("Found {} files belonging to {} directories".format(
    len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    )
)

Found 6 files belonging to 2 directories


In [21]:
#resample noise
command = (
    "for dir in 'ls -1 " + DATASET_NOISE_PATH + "`; do"
    "for file in 'ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate = `ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

512

In [ ]:
#split noise
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels = 1
    )
    